In [17]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Food is vital to our survival and keeps our bodies in a functioning state. Food becomes a necessary component of our body’s functioning the moment we are born",
        metadata={"source": "doc1", "author": "Alice", "date": "2023-01-01"}
    ),
    Document(
        page_content="A balanced diet improves our ability to fend off sickness, boosts our immunity, gives us energy throughout the day, and controls our mood. It further aids in meeting our body’s developmental benchmarks at various growth stages.",
        metadata={"source": "doc2", "author": "Bob", "date": "2023-02-01"}
    ),
    Document(
        page_content="Furthermore, food plays a significant role in fostering global cultural experiences and connections with a diverse range of individuals.",
        metadata={"source": "doc3", "author": "Charlie", "date": "2023-03-01"}
    ),
    Document(
        page_content="Cats are one of the most popular and cherished pets worldwide. Known for their playful nature, independence, and affectionate demeanour, cats have been human companions for centuries. ",
        metadata={"source": "doc4", "author": "Alice", "date": "2023-04-01"}
    ),
    Document(
        page_content="The cat is a domestic animal. Its scientific name is Felis catus. It is a small animal that belongs to the “Felidae” family.",
        metadata={"source": "doc5", "author": "Bob", "date": "2023-05-01"})
]



In [10]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")
os.environ["HUGGINGFACEHUB_API_TOKEN"] = os.getenv("HUGGINGFACEHUB_API_TOKEN")

llm = ChatGroq(
    model="Llama3-8b-8192",
    groq_api_key=groq_api_key)

In [12]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(
    model_name="all-MiniLM-L6-v2")

/opt/miniconda3/envs/practice/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
### vectorstores
from langchain_chroma import Chroma
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings)
vectorstore

In [21]:
vectorstore.similarity_search("What is the dog")

[Document(id='ab488b36-f01c-46c1-87bf-1d0de3a67a61', metadata={'date': '2023-05-01', 'author': 'Bob', 'source': 'doc5'}, page_content='The cat is a domestic animal. Its scientific name is Felis catus. It is a small animal that belongs to the “Felidae” family.'),
 Document(id='2eff14a0-0253-48c6-9f5f-387886d4bb11', metadata={'author': 'Alice', 'date': '2023-04-01', 'source': 'doc4'}, page_content='Cats are one of the most popular and cherished pets worldwide. Known for their playful nature, independence, and affectionate demeanour, cats have been human companions for centuries. '),
 Document(id='40e87538-9633-4ca4-aef3-bb0506b9ccd2', metadata={'date': '2023-04-01', 'author': 'Alice', 'source': 'doc4'}, page_content='This is the fourth document.'),
 Document(id='857c6243-e9f7-498a-8fd1-78ac8fce04c6', metadata={'author': 'Bob', 'source': 'doc5', 'date': '2023-05-01'}, page_content='This is the fifth document.')]

In [22]:
### async
await vectorstore.asimilarity_search("What is the dog")

[Document(id='ab488b36-f01c-46c1-87bf-1d0de3a67a61', metadata={'source': 'doc5', 'date': '2023-05-01', 'author': 'Bob'}, page_content='The cat is a domestic animal. Its scientific name is Felis catus. It is a small animal that belongs to the “Felidae” family.'),
 Document(id='2eff14a0-0253-48c6-9f5f-387886d4bb11', metadata={'date': '2023-04-01', 'source': 'doc4', 'author': 'Alice'}, page_content='Cats are one of the most popular and cherished pets worldwide. Known for their playful nature, independence, and affectionate demeanour, cats have been human companions for centuries. '),
 Document(id='40e87538-9633-4ca4-aef3-bb0506b9ccd2', metadata={'source': 'doc4', 'author': 'Alice', 'date': '2023-04-01'}, page_content='This is the fourth document.'),
 Document(id='857c6243-e9f7-498a-8fd1-78ac8fce04c6', metadata={'date': '2023-05-01', 'source': 'doc5', 'author': 'Bob'}, page_content='This is the fifth document.')]

In [25]:
vectorstore.similarity_search_with_score("What is the cat family called")

[(Document(id='ab488b36-f01c-46c1-87bf-1d0de3a67a61', metadata={'date': '2023-05-01', 'source': 'doc5', 'author': 'Bob'}, page_content='The cat is a domestic animal. Its scientific name is Felis catus. It is a small animal that belongs to the “Felidae” family.'),
  0.6697719097137451),
 (Document(id='2eff14a0-0253-48c6-9f5f-387886d4bb11', metadata={'author': 'Alice', 'source': 'doc4', 'date': '2023-04-01'}, page_content='Cats are one of the most popular and cherished pets worldwide. Known for their playful nature, independence, and affectionate demeanour, cats have been human companions for centuries. '),
  1.0357447862625122),
 (Document(id='09690d2c-98a0-42c2-ac0c-6331ef999330', metadata={'date': '2023-03-01', 'source': 'doc3', 'author': 'Charlie'}, page_content='Furthermore, food plays a significant role in fostering global cultural experiences and connections with a diverse range of individuals.'),
  1.8387198448181152),
 (Document(id='420a1afc-e1b4-487d-a3be-85e8b759cdb4', metadat

In [ ]:
# Retrievers
